## Продуктовая аналитика

**Постановка задачи.**  
Вы пришли работать аналитиком в команду по разработке сайта в компанию, которая занимается продажами продуктов питания. К продакт-менеджеру пришел запрос от Генерального директора компании (ГД) - посчитать эффективность работу продукта (т.е. сайта) за последние два года.  
Оценку необходимо производить через ключевые показатели эффективности:
1. Прирост покупателей (User Growth Rate)
2. Churn Rate And Retaintion Rate
3. Daily Active User (DAU)
4. Monthly Active User (MAU)
5. Наиболее популярные продаваемые товары (Топ-10)
6. Товары с самым высоким доходом
7. Объем выручки

### Описание таблиц
##### sales_orders_items:
    • order_item_id - идентификатор строки заказа (первичный ключ таблицы)
    • fk_product_id - артикул товара
    • fk_order_id - номер заказа
    • ordered_quantity - количество единиц товара в закзе
    • order_quantity_accepted - колиечество единиц товара, доставленных покупателю
    • rate - рейтинг
##### login_logs:
    • login_log_id - идентификатор строки таблицы с логами авторизации пользователей (первичный ключ таблицы)
    • user_id - идентификатор пользователя
    • login_time - дата и время авторизации
##### sales_orders:
    • order_id - номер заказа (первичный ключ таблицы)
    • fk_buyer_id - идентификатор покупателя
    • fk_depot_id - идентификатор склада
    • sales_order_status - статус заказа
    • creation_time - дата и время создания заказа

## Retention Rate

In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Открытие исходных файлов с помощью Pandas и задание им переменных
#pd.read_csv
df_ord_itm = pd.read_csv("sales_orders_items.csv")
df_logs = pd.read_csv("login_logs.csv")
df_ord = pd.read_csv("sales_orders.csv")
df_logs

,login_log_id,user_id,login_time
0,1385328,99160,2021-07-01 00:00:33.0
1,1385329,83272,2021-07-01 00:00:38.0
2,1385330,83272,2021-07-01 00:00:42.0
3,1385331,96640,2021-07-01 00:02:08.0
4,1385332,99160,2021-07-01 00:02:38.0
...,...,...,...
666352,4670509,207632,2022-07-30 23:47:02.0
666353,4670510,207632,2022-07-30 23:49:39.0
666354,4670511,207632,2022-07-30 23:49:43.0
666355,4670512,111788,2022-07-30 23:51:22.0


In [3]:
# Подготовка таблицы login_logs
# Преобразование в дату
# Создание дополнительных столбцов с годом, месяцем, днем

df_logs['login_time'] = df_logs['login_time'].astype('datetime64[ns]')
df_logs['login_year'] =  df_logs['login_time'].dt.to_period("Y")
df_logs['login_month'] = df_logs['login_time'].dt.to_period("M")
df_logs['login_day'] = df_logs['login_time'].dt.to_period("D")
df_logs

,login_log_id,user_id,login_time,login_year,login_month,login_day
0,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01
1,1385329,83272,2021-07-01 00:00:38,2021,2021-07,2021-07-01
2,1385330,83272,2021-07-01 00:00:42,2021,2021-07,2021-07-01
3,1385331,96640,2021-07-01 00:02:08,2021,2021-07,2021-07-01
4,1385332,99160,2021-07-01 00:02:38,2021,2021-07,2021-07-01
...,...,...,...,...,...,...
666352,4670509,207632,2022-07-30 23:47:02,2022,2022-07,2022-07-30
666353,4670510,207632,2022-07-30 23:49:39,2022,2022-07,2022-07-30
666354,4670511,207632,2022-07-30 23:49:43,2022,2022-07,2022-07-30
666355,4670512,111788,2022-07-30 23:51:22,2022,2022-07,2022-07-30


In [4]:
# Подготовка таблицы sales_orders
#dt.to_period("M")
df_ord['creation_time'] = df_ord['creation_time'].astype('datetime64[ns]')
df_ord['creation_year'] =  df_ord['creation_time'].dt.to_period("Y")
df_ord['creation_month'] = df_ord['creation_time'].dt.to_period("M")
df_ord['creation_day'] = df_ord['creation_time'].dt.to_period("D")
df_ord

,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day
0,119874,82228,5,Rejected,2021-07-01 00:35:28,2021,2021-07,2021-07-01
1,119877,40476,1,Rejected,2021-07-01 02:02:43,2021,2021-07,2021-07-01
2,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01
3,119883,97616,4,Rejected,2021-07-01 02:56:00,2021,2021-07,2021-07-01
4,119886,12440,1,Rejected,2021-07-01 03:11:31,2021,2021-07,2021-07-01
...,...,...,...,...,...,...,...,...
13625,318960,63380,10,Rejected,2022-07-30 16:02:58,2022,2022-07,2022-07-30
13626,318963,69544,5,Rejected,2022-07-30 16:30:26,2022,2022-07,2022-07-30
13627,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30
13628,318969,122360,2,Rejected,2022-07-30 17:53:54,2022,2022-07,2022-07-30


In [5]:
#Разделение таблицы login_logs по годам
df_logs_2021 = df_logs[df_logs['login_year'] == "2021"]
df_logs_2022 = df_logs[df_logs['login_year'] == "2022"]
df_logs_2021

,login_log_id,user_id,login_time,login_year,login_month,login_day
0,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01
1,1385329,83272,2021-07-01 00:00:38,2021,2021-07,2021-07-01
2,1385330,83272,2021-07-01 00:00:42,2021,2021-07,2021-07-01
3,1385331,96640,2021-07-01 00:02:08,2021,2021-07,2021-07-01
4,1385332,99160,2021-07-01 00:02:38,2021,2021-07,2021-07-01
...,...,...,...,...,...,...
271235,1656563,98452,2021-07-30 23:45:53,2021,2021-07,2021-07-30
271236,1656564,98452,2021-07-30 23:51:16,2021,2021-07,2021-07-30
271237,1656565,35732,2021-07-30 23:53:48,2021,2021-07,2021-07-30
271238,1656566,35732,2021-07-30 23:53:54,2021,2021-07,2021-07-30


In [6]:
#Разделение таблицы sales_orders по годам
df_ord_2021 = df_ord[df_ord['creation_year'] == "2021"]
df_ord_2022 = df_ord[df_ord['creation_year'] == "2022"]
df_ord_2021

,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day
0,119874,82228,5,Rejected,2021-07-01 00:35:28,2021,2021-07,2021-07-01
1,119877,40476,1,Rejected,2021-07-01 02:02:43,2021,2021-07,2021-07-01
2,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01
3,119883,97616,4,Rejected,2021-07-01 02:56:00,2021,2021-07,2021-07-01
4,119886,12440,1,Rejected,2021-07-01 03:11:31,2021,2021-07,2021-07-01
...,...,...,...,...,...,...,...,...
6211,138507,76516,8,Rejected,2021-07-30 17:45:38,2021,2021-07,2021-07-30
6212,138510,58192,1,Rejected,2021-07-30 18:35:37,2021,2021-07,2021-07-30
6213,138513,45296,1,Rejected,2021-07-30 23:08:06,2021,2021-07,2021-07-30
6214,138516,45296,1,Rejected,2021-07-30 23:12:30,2021,2021-07,2021-07-30


In [7]:
#Объединяем данные по годам
#merge(on = 'user_id', how = 'left')
df_2021 = df_logs_2021.merge(df_ord_2021, left_on = 'user_id', right_on = 'fk_buyer_id', how = 'left' )
df_2022 = df_logs_2022.merge(df_ord_2022, left_on = 'user_id', right_on = 'fk_buyer_id', how = 'left' )
df_2021

,login_log_id,user_id,login_time,login_year,login_month,login_day,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day
0,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,126408.0,99160.0,9.0,Shipped,2021-07-11 22:59:43,2021,2021-07,2021-07-11
1,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138030.0,99160.0,9.0,Rejected,2021-07-30 07:43:51,2021,2021-07,2021-07-30
2,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138039.0,99160.0,9.0,Rejected,2021-07-30 07:47:29,2021,2021-07,2021-07-30
3,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138078.0,99160.0,9.0,Shipped,2021-07-30 08:24:19,2021,2021-07,2021-07-30
4,1385329,83272,2021-07-01 00:00:38,2021,2021-07,2021-07-01,121407.0,83272.0,5.0,Rejected,2021-07-04 05:37:18,2021,2021-07,2021-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831926,1656566,35732,2021-07-30 23:53:54,2021,2021-07,2021-07-30,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
831927,1656567,85780,2021-07-30 23:55:19,2021,2021-07,2021-07-30,121554.0,85780.0,3.0,Rejected,2021-07-04 11:49:25,2021,2021-07,2021-07-04
831928,1656567,85780,2021-07-30 23:55:19,2021,2021-07,2021-07-30,121608.0,85780.0,3.0,Rejected,2021-07-04 17:41:04,2021,2021-07,2021-07-04
831929,1656567,85780,2021-07-30 23:55:19,2021,2021-07,2021-07-30,125400.0,85780.0,3.0,Rejected,2021-07-09 23:28:33,2021,2021-07,2021-07-09


In [8]:
#Отсекаем в объединенных таблицах строки, где время логина меньше создания заказа
df_2021 = df_2021[df_2021['creation_time'] > df_2021['login_time']]
df_2022 = df_2022[df_2022['creation_time'] > df_2022['login_time']]
df_2021

,login_log_id,user_id,login_time,login_year,login_month,login_day,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day
0,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,126408.0,99160.0,9.0,Shipped,2021-07-11 22:59:43,2021,2021-07,2021-07-11
1,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138030.0,99160.0,9.0,Rejected,2021-07-30 07:43:51,2021,2021-07,2021-07-30
2,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138039.0,99160.0,9.0,Rejected,2021-07-30 07:47:29,2021,2021-07,2021-07-30
3,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138078.0,99160.0,9.0,Shipped,2021-07-30 08:24:19,2021,2021-07,2021-07-30
4,1385329,83272,2021-07-01 00:00:38,2021,2021-07,2021-07-01,121407.0,83272.0,5.0,Rejected,2021-07-04 05:37:18,2021,2021-07,2021-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831871,1656544,45296,2021-07-30 23:01:33,2021,2021-07,2021-07-30,138516.0,45296.0,1.0,Rejected,2021-07-30 23:12:30,2021,2021-07,2021-07-30
831872,1656544,45296,2021-07-30 23:01:33,2021,2021-07,2021-07-30,138519.0,45296.0,1.0,Rejected,2021-07-30 23:17:29,2021,2021-07,2021-07-30
831876,1656547,45296,2021-07-30 23:09:14,2021,2021-07,2021-07-30,138516.0,45296.0,1.0,Rejected,2021-07-30 23:12:30,2021,2021-07,2021-07-30
831877,1656547,45296,2021-07-30 23:09:14,2021,2021-07,2021-07-30,138519.0,45296.0,1.0,Rejected,2021-07-30 23:17:29,2021,2021-07,2021-07-30


In [9]:
df_2021

,login_log_id,user_id,login_time,login_year,login_month,login_day,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day
0,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,126408.0,99160.0,9.0,Shipped,2021-07-11 22:59:43,2021,2021-07,2021-07-11
1,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138030.0,99160.0,9.0,Rejected,2021-07-30 07:43:51,2021,2021-07,2021-07-30
2,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138039.0,99160.0,9.0,Rejected,2021-07-30 07:47:29,2021,2021-07,2021-07-30
3,1385328,99160,2021-07-01 00:00:33,2021,2021-07,2021-07-01,138078.0,99160.0,9.0,Shipped,2021-07-30 08:24:19,2021,2021-07,2021-07-30
4,1385329,83272,2021-07-01 00:00:38,2021,2021-07,2021-07-01,121407.0,83272.0,5.0,Rejected,2021-07-04 05:37:18,2021,2021-07,2021-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831871,1656544,45296,2021-07-30 23:01:33,2021,2021-07,2021-07-30,138516.0,45296.0,1.0,Rejected,2021-07-30 23:12:30,2021,2021-07,2021-07-30
831872,1656544,45296,2021-07-30 23:01:33,2021,2021-07,2021-07-30,138519.0,45296.0,1.0,Rejected,2021-07-30 23:17:29,2021,2021-07,2021-07-30
831876,1656547,45296,2021-07-30 23:09:14,2021,2021-07,2021-07-30,138516.0,45296.0,1.0,Rejected,2021-07-30 23:12:30,2021,2021-07,2021-07-30
831877,1656547,45296,2021-07-30 23:09:14,2021,2021-07,2021-07-30,138519.0,45296.0,1.0,Rejected,2021-07-30 23:17:29,2021,2021-07,2021-07-30


In [10]:
#Конвертируем столбцы с днём в формат даты
# dt.to_timestamp()
import datetime as dt
df_2021['creation_day'] = df_2021['creation_day'].dt.to_timestamp()
df_2021['login_day'] = df_2021['login_day'].dt.to_timestamp()
df_2022['creation_day'] = df_2022['creation_day'].dt.to_timestamp()
df_2022['login_day'] = df_2022['login_day'].dt.to_timestamp()



In [11]:
# Вычисляем разницу между датами
df_2021['lifetime'] = df_2021['creation_day'] - df_2021['login_day']
df_2021['lifetime']

0        10 days
1        29 days
2        29 days
3        29 days
4         3 days
           ...  
831871    0 days
831872    0 days
831876    0 days
831877    0 days
831888    0 days
Name: lifetime, Length: 325232, dtype: timedelta64[ns]

In [12]:
df_2022['lifetime'] = df_2022['creation_day'] - df_2022['login_day']
df_2022['lifetime']

0          0 days
1         22 days
3          0 days
4          4 days
5          4 days
            ...  
1100730    0 days
1100731    0 days
1100806    0 days
1100985    0 days
1101019    0 days
Name: lifetime, Length: 434007, dtype: timedelta64[ns]

In [13]:
# Переводим разницу в дни
# dt.days
df_2021['lifetime'] = df_2021['lifetime'].dt.days

In [14]:
df_2021['lifetime']

0         10
1         29
2         29
3         29
4          3
          ..
831871     0
831872     0
831876     0
831877     0
831888     0
Name: lifetime, Length: 325232, dtype: int64

In [15]:
df_2022['lifetime'] = df_2022['lifetime'].dt.days
df_2022['lifetime']

0           0
1          22
3           0
4           4
5           4
           ..
1100730     0
1100731     0
1100806     0
1100985     0
1101019     0
Name: lifetime, Length: 434007, dtype: int64

In [16]:
#Строим матрицу когорт
#pivot_table
test_table = pd.pivot_table(df_2021, values = 'user_id', index = 'login_day', columns = 'lifetime', aggfunc= "nunique").fillna(0).reset_index()
test_table

lifetime,login_day,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,2021-07-01,110.0,60.0,49.0,26.0,71.0,59.0,63.0,61.0,57.0,...,49.0,32.0,40.0,30.0,18.0,48.0,48.0,45.0,34.0,48.0
1,2021-07-02,127.0,55.0,27.0,77.0,61.0,59.0,55.0,61.0,60.0,...,30.0,32.0,33.0,13.0,47.0,48.0,42.0,34.0,48.0,0.0
2,2021-07-03,116.0,27.0,81.0,59.0,49.0,59.0,48.0,58.0,20.0,...,36.0,29.0,15.0,48.0,36.0,39.0,34.0,44.0,0.0,0.0
3,2021-07-04,61.0,70.0,44.0,45.0,48.0,53.0,48.0,18.0,50.0,...,32.0,13.0,42.0,30.0,40.0,30.0,44.0,0.0,0.0,0.0
4,2021-07-05,187.0,79.0,71.0,62.0,59.0,72.0,30.0,76.0,52.0,...,13.0,48.0,47.0,42.0,39.0,59.0,0.0,0.0,0.0,0.0
5,2021-07-06,173.0,83.0,61.0,67.0,62.0,26.0,74.0,41.0,52.0,...,53.0,48.0,42.0,37.0,43.0,0.0,0.0,0.0,0.0,0.0
6,2021-07-07,179.0,86.0,67.0,70.0,26.0,78.0,52.0,48.0,63.0,...,49.0,48.0,39.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2021-07-08,173.0,82.0,77.0,27.0,80.0,52.0,50.0,48.0,49.0,...,46.0,42.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2021-07-09,165.0,80.0,32.0,83.0,46.0,51.0,53.0,44.0,40.0,...,37.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2021-07-10,168.0,34.0,87.0,49.0,55.0,58.0,37.0,48.0,21.0,...,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#Выделяем численность когорты (уникальных пользователей)
temp_table = df_logs_2021[['login_day', 'user_id']]
temp_table['login_day'] = temp_table['login_day'].dt.to_timestamp()

In [18]:
unique_table = temp_table.groupby(by = 'login_day').nunique().reset_index()
unique_table


,login_day,user_id
0,2021-07-01,1563
1,2021-07-02,1519
2,2021-07-03,1485
3,2021-07-04,1128
4,2021-07-05,1536
5,2021-07-06,1507
6,2021-07-07,1526
7,2021-07-08,1586
8,2021-07-09,1561
9,2021-07-10,1551


In [19]:
# Соединим когорты с численностью
cohorts_2021 = unique_table.merge(test_table, on = 'login_day', how = 'left')

In [20]:
test_table_2 = pd.pivot_table(df_2022, values = 'user_id', index = 'login_day', columns = 'lifetime', aggfunc= "nunique").fillna(0).reset_index()
test_table_2

lifetime,login_day,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,2022-07-01,143.0,93.0,37.0,93.0,78.0,62.0,70.0,71.0,44.0,...,64.0,74.0,56.0,23.0,57.0,48.0,59.0,62.0,63.0,56.0
1,2022-07-02,182.0,43.0,119.0,97.0,64.0,64.0,76.0,47.0,21.0,...,90.0,57.0,24.0,66.0,55.0,53.0,69.0,68.0,56.0,0.0
2,2022-07-03,90.0,110.0,83.0,62.0,60.0,71.0,49.0,19.0,67.0,...,48.0,20.0,58.0,38.0,61.0,62.0,56.0,46.0,0.0,0.0
3,2022-07-04,257.0,131.0,87.0,85.0,92.0,57.0,27.0,92.0,77.0,...,26.0,76.0,56.0,64.0,68.0,72.0,61.0,0.0,0.0,0.0
4,2022-07-05,222.0,99.0,92.0,100.0,46.0,26.0,89.0,87.0,81.0,...,83.0,59.0,62.0,70.0,74.0,54.0,0.0,0.0,0.0,0.0
5,2022-07-06,167.0,116.0,89.0,53.0,22.0,75.0,80.0,69.0,82.0,...,50.0,61.0,59.0,68.0,53.0,0.0,0.0,0.0,0.0,0.0
6,2022-07-07,205.0,120.0,72.0,27.0,89.0,90.0,82.0,95.0,82.0,...,76.0,68.0,77.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2022-07-08,197.0,74.0,26.0,104.0,103.0,91.0,87.0,76.0,81.0,...,66.0,74.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2022-07-09,149.0,27.0,116.0,93.0,86.0,89.0,84.0,82.0,31.0,...,69.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2022-07-10,57.0,87.0,68.0,69.0,72.0,62.0,66.0,19.0,54.0,...,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
temp_table_2 = df_logs_2022[['login_day', 'user_id']]
temp_table_2['login_day'] = temp_table_2['login_day'].dt.to_timestamp()

In [22]:
unique_table_2 = temp_table_2.groupby(by = 'login_day').nunique().reset_index()
unique_table_2

,login_day,user_id
0,2022-07-01,2073
1,2022-07-02,2329
2,2022-07-03,2010
3,2022-07-04,2701
4,2022-07-05,2556
5,2022-07-06,2230
6,2022-07-07,2550
7,2022-07-08,2397
8,2022-07-09,2276
9,2022-07-10,1572


In [23]:
cohorts_2022 = unique_table_2.merge(test_table_2, on = 'login_day', how = 'left')

In [24]:
cohorts_2022

,login_day,user_id,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,2022-07-01,2073,143.0,93.0,37.0,93.0,78.0,62.0,70.0,71.0,...,64.0,74.0,56.0,23.0,57.0,48.0,59.0,62.0,63.0,56.0
1,2022-07-02,2329,182.0,43.0,119.0,97.0,64.0,64.0,76.0,47.0,...,90.0,57.0,24.0,66.0,55.0,53.0,69.0,68.0,56.0,0.0
2,2022-07-03,2010,90.0,110.0,83.0,62.0,60.0,71.0,49.0,19.0,...,48.0,20.0,58.0,38.0,61.0,62.0,56.0,46.0,0.0,0.0
3,2022-07-04,2701,257.0,131.0,87.0,85.0,92.0,57.0,27.0,92.0,...,26.0,76.0,56.0,64.0,68.0,72.0,61.0,0.0,0.0,0.0
4,2022-07-05,2556,222.0,99.0,92.0,100.0,46.0,26.0,89.0,87.0,...,83.0,59.0,62.0,70.0,74.0,54.0,0.0,0.0,0.0,0.0
5,2022-07-06,2230,167.0,116.0,89.0,53.0,22.0,75.0,80.0,69.0,...,50.0,61.0,59.0,68.0,53.0,0.0,0.0,0.0,0.0,0.0
6,2022-07-07,2550,205.0,120.0,72.0,27.0,89.0,90.0,82.0,95.0,...,76.0,68.0,77.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2022-07-08,2397,197.0,74.0,26.0,104.0,103.0,91.0,87.0,76.0,...,66.0,74.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2022-07-09,2276,149.0,27.0,116.0,93.0,86.0,89.0,84.0,82.0,...,69.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2022-07-10,1572,57.0,87.0,68.0,69.0,72.0,62.0,66.0,19.0,...,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#Меняем тип данных в первом столбце
cohorts_2021['user_id'] = cohorts_2021['user_id'].apply(float)
cohorts_2022['user_id'] = cohorts_2022['user_id'].apply(float)

In [26]:
#Засунем в индекс первый столбец с датой
retention_2021 = cohorts_2021
retention_2021 = retention_2021.set_index('login_day')

In [27]:
#Наконец считаем Retention Rate
# round
# div
retention_2021 = round(retention_2021.div(retention_2021.user_id, axis = 0)* 100, 2)

In [28]:
retention_2021

,user_id,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
login_day,,,,,,,,,,,,,,,,,,,,,
2021-07-01,100.0,7.04,3.84,3.13,1.66,4.54,3.77,4.03,3.90,3.65,...,3.13,2.05,2.56,1.92,1.15,3.07,3.07,2.88,2.18,3.07
2021-07-02,100.0,8.36,3.62,1.78,5.07,4.02,3.88,3.62,4.02,3.95,...,1.97,2.11,2.17,0.86,3.09,3.16,2.76,2.24,3.16,0.00
2021-07-03,100.0,7.81,1.82,5.45,3.97,3.30,3.97,3.23,3.91,1.35,...,2.42,1.95,1.01,3.23,2.42,2.63,2.29,2.96,0.00,0.00
2021-07-04,100.0,5.41,6.21,3.90,3.99,4.26,4.70,4.26,1.60,4.43,...,2.84,1.15,3.72,2.66,3.55,2.66,3.90,0.00,0.00,0.00
2021-07-05,100.0,12.17,5.14,4.62,4.04,3.84,4.69,1.95,4.95,3.39,...,0.85,3.12,3.06,2.73,2.54,3.84,0.00,0.00,0.00,0.00
2021-07-06,100.0,11.48,5.51,4.05,4.45,4.11,1.73,4.91,2.72,3.45,...,3.52,3.19,2.79,2.46,2.85,0.00,0.00,0.00,0.00,0.00
2021-07-07,100.0,11.73,5.64,4.39,4.59,1.70,5.11,3.41,3.15,4.13,...,3.21,3.15,2.56,3.67,0.00,0.00,0.00,0.00,0.00,0.00
2021-07-08,100.0,10.91,5.17,4.85,1.70,5.04,3.28,3.15,3.03,3.09,...,2.90,2.65,3.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2021-07-09,100.0,10.57,5.12,2.05,5.32,2.95,3.27,3.40,2.82,2.56,...,2.37,2.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [29]:
retention_2022 = cohorts_2022
retention_2022 = retention_2022.set_index('login_day')

In [30]:
retention_2022 = round(retention_2022.div(retention_2022.user_id, axis = 0) * 100, 2)

In [31]:
#Считаем Сhurn Rate
churn_rate_2021 = retention_2021 - 100
churn_rate_2021

,user_id,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
login_day,,,,,,,,,,,,,,,,,,,,,
2021-07-01,0.0,-92.96,-96.16,-96.87,-98.34,-95.46,-96.23,-95.97,-96.10,-96.35,...,-96.87,-97.95,-97.44,-98.08,-98.85,-96.93,-96.93,-97.12,-97.82,-96.93
2021-07-02,0.0,-91.64,-96.38,-98.22,-94.93,-95.98,-96.12,-96.38,-95.98,-96.05,...,-98.03,-97.89,-97.83,-99.14,-96.91,-96.84,-97.24,-97.76,-96.84,-100.00
2021-07-03,0.0,-92.19,-98.18,-94.55,-96.03,-96.70,-96.03,-96.77,-96.09,-98.65,...,-97.58,-98.05,-98.99,-96.77,-97.58,-97.37,-97.71,-97.04,-100.00,-100.00
2021-07-04,0.0,-94.59,-93.79,-96.10,-96.01,-95.74,-95.30,-95.74,-98.40,-95.57,...,-97.16,-98.85,-96.28,-97.34,-96.45,-97.34,-96.10,-100.00,-100.00,-100.00
2021-07-05,0.0,-87.83,-94.86,-95.38,-95.96,-96.16,-95.31,-98.05,-95.05,-96.61,...,-99.15,-96.88,-96.94,-97.27,-97.46,-96.16,-100.00,-100.00,-100.00,-100.00
2021-07-06,0.0,-88.52,-94.49,-95.95,-95.55,-95.89,-98.27,-95.09,-97.28,-96.55,...,-96.48,-96.81,-97.21,-97.54,-97.15,-100.00,-100.00,-100.00,-100.00,-100.00
2021-07-07,0.0,-88.27,-94.36,-95.61,-95.41,-98.30,-94.89,-96.59,-96.85,-95.87,...,-96.79,-96.85,-97.44,-96.33,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00
2021-07-08,0.0,-89.09,-94.83,-95.15,-98.30,-94.96,-96.72,-96.85,-96.97,-96.91,...,-97.10,-97.35,-96.66,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00
2021-07-09,0.0,-89.43,-94.88,-97.95,-94.68,-97.05,-96.73,-96.60,-97.18,-97.44,...,-97.63,-97.25,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00


In [32]:
churn_rate_2022 = retention_2022 - 100
churn_rate_2022

,user_id,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
login_day,,,,,,,,,,,,,,,,,,,,,
2022-07-01,0.0,-93.10,-95.51,-98.22,-95.51,-96.24,-97.01,-96.62,-96.58,-97.88,...,-96.91,-96.43,-97.30,-98.89,-97.25,-97.68,-97.15,-97.01,-96.96,-97.3
2022-07-02,0.0,-92.19,-98.15,-94.89,-95.84,-97.25,-97.25,-96.74,-97.98,-99.10,...,-96.14,-97.55,-98.97,-97.17,-97.64,-97.72,-97.04,-97.08,-97.60,-100.0
2022-07-03,0.0,-95.52,-94.53,-95.87,-96.92,-97.01,-96.47,-97.56,-99.05,-96.67,...,-97.61,-99.00,-97.11,-98.11,-96.97,-96.92,-97.21,-97.71,-100.00,-100.0
2022-07-04,0.0,-90.49,-95.15,-96.78,-96.85,-96.59,-97.89,-99.00,-96.59,-97.15,...,-99.04,-97.19,-97.93,-97.63,-97.48,-97.33,-97.74,-100.00,-100.00,-100.0
2022-07-05,0.0,-91.31,-96.13,-96.40,-96.09,-98.20,-98.98,-96.52,-96.60,-96.83,...,-96.75,-97.69,-97.57,-97.26,-97.10,-97.89,-100.00,-100.00,-100.00,-100.0
2022-07-06,0.0,-92.51,-94.80,-96.01,-97.62,-99.01,-96.64,-96.41,-96.91,-96.32,...,-97.76,-97.26,-97.35,-96.95,-97.62,-100.00,-100.00,-100.00,-100.00,-100.0
2022-07-07,0.0,-91.96,-95.29,-97.18,-98.94,-96.51,-96.47,-96.78,-96.27,-96.78,...,-97.02,-97.33,-96.98,-97.57,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0
2022-07-08,0.0,-91.78,-96.91,-98.92,-95.66,-95.70,-96.20,-96.37,-96.83,-96.62,...,-97.25,-96.91,-97.29,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0
2022-07-09,0.0,-93.45,-98.81,-94.90,-95.91,-96.22,-96.09,-96.31,-96.40,-98.64,...,-96.97,-97.54,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0


In [33]:
#Делаем красивую табличку для 2021 года и смотрим
retention_2021 = retention_2021.reset_index().\
            rename(columns={'login_day':'День посещения','user_id':'Пользователи'})
retention_2021

,День посещения,Пользователи,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,2021-07-01,100.0,7.04,3.84,3.13,1.66,4.54,3.77,4.03,3.90,...,3.13,2.05,2.56,1.92,1.15,3.07,3.07,2.88,2.18,3.07
1,2021-07-02,100.0,8.36,3.62,1.78,5.07,4.02,3.88,3.62,4.02,...,1.97,2.11,2.17,0.86,3.09,3.16,2.76,2.24,3.16,0.00
2,2021-07-03,100.0,7.81,1.82,5.45,3.97,3.30,3.97,3.23,3.91,...,2.42,1.95,1.01,3.23,2.42,2.63,2.29,2.96,0.00,0.00
3,2021-07-04,100.0,5.41,6.21,3.90,3.99,4.26,4.70,4.26,1.60,...,2.84,1.15,3.72,2.66,3.55,2.66,3.90,0.00,0.00,0.00
4,2021-07-05,100.0,12.17,5.14,4.62,4.04,3.84,4.69,1.95,4.95,...,0.85,3.12,3.06,2.73,2.54,3.84,0.00,0.00,0.00,0.00
5,2021-07-06,100.0,11.48,5.51,4.05,4.45,4.11,1.73,4.91,2.72,...,3.52,3.19,2.79,2.46,2.85,0.00,0.00,0.00,0.00,0.00
6,2021-07-07,100.0,11.73,5.64,4.39,4.59,1.70,5.11,3.41,3.15,...,3.21,3.15,2.56,3.67,0.00,0.00,0.00,0.00,0.00,0.00
7,2021-07-08,100.0,10.91,5.17,4.85,1.70,5.04,3.28,3.15,3.03,...,2.90,2.65,3.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2021-07-09,100.0,10.57,5.12,2.05,5.32,2.95,3.27,3.40,2.82,...,2.37,2.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,2021-07-10,100.0,10.83,2.19,5.61,3.16,3.55,3.74,2.39,3.09,...,3.61,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [34]:
#Делаем красивую табличку для 2022 года и смотрим
retention_2022 = retention_2022.reset_index().\
            rename(columns={'login_day':'День посещения','user_id':'Пользователи'})
retention_2022

,День посещения,Пользователи,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,2022-07-01,100.0,6.90,4.49,1.78,4.49,3.76,2.99,3.38,3.42,...,3.09,3.57,2.70,1.11,2.75,2.32,2.85,2.99,3.04,2.7
1,2022-07-02,100.0,7.81,1.85,5.11,4.16,2.75,2.75,3.26,2.02,...,3.86,2.45,1.03,2.83,2.36,2.28,2.96,2.92,2.40,0.0
2,2022-07-03,100.0,4.48,5.47,4.13,3.08,2.99,3.53,2.44,0.95,...,2.39,1.00,2.89,1.89,3.03,3.08,2.79,2.29,0.00,0.0
3,2022-07-04,100.0,9.51,4.85,3.22,3.15,3.41,2.11,1.00,3.41,...,0.96,2.81,2.07,2.37,2.52,2.67,2.26,0.00,0.00,0.0
4,2022-07-05,100.0,8.69,3.87,3.60,3.91,1.80,1.02,3.48,3.40,...,3.25,2.31,2.43,2.74,2.90,2.11,0.00,0.00,0.00,0.0
5,2022-07-06,100.0,7.49,5.20,3.99,2.38,0.99,3.36,3.59,3.09,...,2.24,2.74,2.65,3.05,2.38,0.00,0.00,0.00,0.00,0.0
6,2022-07-07,100.0,8.04,4.71,2.82,1.06,3.49,3.53,3.22,3.73,...,2.98,2.67,3.02,2.43,0.00,0.00,0.00,0.00,0.00,0.0
7,2022-07-08,100.0,8.22,3.09,1.08,4.34,4.30,3.80,3.63,3.17,...,2.75,3.09,2.71,0.00,0.00,0.00,0.00,0.00,0.00,0.0
8,2022-07-09,100.0,6.55,1.19,5.10,4.09,3.78,3.91,3.69,3.60,...,3.03,2.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
9,2022-07-10,100.0,3.63,5.53,4.33,4.39,4.58,3.94,4.20,1.21,...,2.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [35]:
#Делаем красивую табличку для 2021 года и смотрим
churn_rate_2021 = churn_rate_2021.reset_index().\
            rename(columns={'login_day':'День посещения','user_id':'Пользователи'})
churn_rate_2021

,День посещения,Пользователи,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,2021-07-01,0.0,-92.96,-96.16,-96.87,-98.34,-95.46,-96.23,-95.97,-96.10,...,-96.87,-97.95,-97.44,-98.08,-98.85,-96.93,-96.93,-97.12,-97.82,-96.93
1,2021-07-02,0.0,-91.64,-96.38,-98.22,-94.93,-95.98,-96.12,-96.38,-95.98,...,-98.03,-97.89,-97.83,-99.14,-96.91,-96.84,-97.24,-97.76,-96.84,-100.00
2,2021-07-03,0.0,-92.19,-98.18,-94.55,-96.03,-96.70,-96.03,-96.77,-96.09,...,-97.58,-98.05,-98.99,-96.77,-97.58,-97.37,-97.71,-97.04,-100.00,-100.00
3,2021-07-04,0.0,-94.59,-93.79,-96.10,-96.01,-95.74,-95.30,-95.74,-98.40,...,-97.16,-98.85,-96.28,-97.34,-96.45,-97.34,-96.10,-100.00,-100.00,-100.00
4,2021-07-05,0.0,-87.83,-94.86,-95.38,-95.96,-96.16,-95.31,-98.05,-95.05,...,-99.15,-96.88,-96.94,-97.27,-97.46,-96.16,-100.00,-100.00,-100.00,-100.00
5,2021-07-06,0.0,-88.52,-94.49,-95.95,-95.55,-95.89,-98.27,-95.09,-97.28,...,-96.48,-96.81,-97.21,-97.54,-97.15,-100.00,-100.00,-100.00,-100.00,-100.00
6,2021-07-07,0.0,-88.27,-94.36,-95.61,-95.41,-98.30,-94.89,-96.59,-96.85,...,-96.79,-96.85,-97.44,-96.33,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00
7,2021-07-08,0.0,-89.09,-94.83,-95.15,-98.30,-94.96,-96.72,-96.85,-96.97,...,-97.10,-97.35,-96.66,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00
8,2021-07-09,0.0,-89.43,-94.88,-97.95,-94.68,-97.05,-96.73,-96.60,-97.18,...,-97.63,-97.25,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00
9,2021-07-10,0.0,-89.17,-97.81,-94.39,-96.84,-96.45,-96.26,-97.61,-96.91,...,-96.39,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00


In [36]:
#Делаем красивую табличку для 2022 года и смотрим
churn_rate_2022 = churn_rate_2022.reset_index().\
            rename(columns={'login_day':'День посещения','user_id':'Пользователи'})
churn_rate_2022

,День посещения,Пользователи,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,2022-07-01,0.0,-93.10,-95.51,-98.22,-95.51,-96.24,-97.01,-96.62,-96.58,...,-96.91,-96.43,-97.30,-98.89,-97.25,-97.68,-97.15,-97.01,-96.96,-97.3
1,2022-07-02,0.0,-92.19,-98.15,-94.89,-95.84,-97.25,-97.25,-96.74,-97.98,...,-96.14,-97.55,-98.97,-97.17,-97.64,-97.72,-97.04,-97.08,-97.60,-100.0
2,2022-07-03,0.0,-95.52,-94.53,-95.87,-96.92,-97.01,-96.47,-97.56,-99.05,...,-97.61,-99.00,-97.11,-98.11,-96.97,-96.92,-97.21,-97.71,-100.00,-100.0
3,2022-07-04,0.0,-90.49,-95.15,-96.78,-96.85,-96.59,-97.89,-99.00,-96.59,...,-99.04,-97.19,-97.93,-97.63,-97.48,-97.33,-97.74,-100.00,-100.00,-100.0
4,2022-07-05,0.0,-91.31,-96.13,-96.40,-96.09,-98.20,-98.98,-96.52,-96.60,...,-96.75,-97.69,-97.57,-97.26,-97.10,-97.89,-100.00,-100.00,-100.00,-100.0
5,2022-07-06,0.0,-92.51,-94.80,-96.01,-97.62,-99.01,-96.64,-96.41,-96.91,...,-97.76,-97.26,-97.35,-96.95,-97.62,-100.00,-100.00,-100.00,-100.00,-100.0
6,2022-07-07,0.0,-91.96,-95.29,-97.18,-98.94,-96.51,-96.47,-96.78,-96.27,...,-97.02,-97.33,-96.98,-97.57,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0
7,2022-07-08,0.0,-91.78,-96.91,-98.92,-95.66,-95.70,-96.20,-96.37,-96.83,...,-97.25,-96.91,-97.29,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0
8,2022-07-09,0.0,-93.45,-98.81,-94.90,-95.91,-96.22,-96.09,-96.31,-96.40,...,-96.97,-97.54,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0
9,2022-07-10,0.0,-96.37,-94.47,-95.67,-95.61,-95.42,-96.06,-95.80,-98.79,...,-97.71,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.00,-100.0


In [37]:
# Month Active Users (MAU)
temp_table = df_logs[['login_month', 'user_id']]
temp_table['login_month'] = temp_table['login_month'].dt.to_timestamp()

In [38]:
unique_table = temp_table.groupby(by = 'login_month').nunique().reset_index()
unique_table

,login_month,user_id
0,2021-07-01,10867
1,2022-07-01,13022


In [56]:
#Прирост пользователей 2021
temp_table = df_ord_2021[['creation_day', 'fk_buyer_id']]
temp_table['creation_day'] = temp_table['creation_day'].dt.to_timestamp()
temp_table

,creation_day,fk_buyer_id
0,2021-07-01,82228
1,2021-07-01,40476
2,2021-07-01,41728
3,2021-07-01,97616
4,2021-07-01,12440
...,...,...
6211,2021-07-30,76516
6212,2021-07-30,58192
6213,2021-07-30,45296
6214,2021-07-30,45296


In [57]:
unique_table = temp_table.groupby(by = 'creation_day').nunique().reset_index()
unique_table['delta'] = unique_table['fk_buyer_id'].diff().fillna('-')
unique_table['lag1'] = unique_table['fk_buyer_id'].shift(1)
unique_table['Prirost'] = (unique_table['delta'] / unique_table['lag1']) * 100
unique_table

,creation_day,fk_buyer_id,delta,lag1,Prirost
0,2021-07-01,110,-,NaN,NaN
1,2021-07-02,127,17.0,110.0,15.454545
2,2021-07-03,116,-11.0,127.0,-8.661417
3,2021-07-04,61,-55.0,116.0,-47.413793
4,2021-07-05,187,126.0,61.0,206.557377
5,2021-07-06,173,-14.0,187.0,-7.486631
6,2021-07-07,180,7.0,173.0,4.046243
7,2021-07-08,173,-7.0,180.0,-3.888889
8,2021-07-09,165,-8.0,173.0,-4.624277
9,2021-07-10,168,3.0,165.0,1.818182


In [58]:
#Прирост пользователей 2022
temp_table = df_ord_2022[['creation_day', 'fk_buyer_id']]
temp_table['creation_day'] = temp_table['creation_day'].dt.to_timestamp()
temp_table

,creation_day,fk_buyer_id
6216,2022-07-01,71616
6217,2022-07-01,83936
6218,2022-07-01,179824
6219,2022-07-01,219672
6220,2022-07-01,179824
...,...,...
13625,2022-07-30,63380
13626,2022-07-30,69544
13627,2022-07-30,167048
13628,2022-07-30,122360


In [59]:
unique_table = temp_table.groupby(by = 'creation_day').nunique().reset_index()
unique_table['delta'] = unique_table['fk_buyer_id'].diff().fillna('-')
unique_table['lag1'] = unique_table['fk_buyer_id'].shift(1)
unique_table['Prirost'] = (unique_table['delta'] / unique_table['lag1']) * 100
unique_table

,creation_day,fk_buyer_id,delta,lag1,Prirost
0,2022-07-01,143,-,NaN,NaN
1,2022-07-02,182,39.0,143.0,27.272727
2,2022-07-03,90,-92.0,182.0,-50.549451
3,2022-07-04,257,167.0,90.0,185.555556
4,2022-07-05,222,-35.0,257.0,-13.618677
5,2022-07-06,167,-55.0,222.0,-24.774775
6,2022-07-07,205,38.0,167.0,22.754491
7,2022-07-08,197,-8.0,205.0,-3.902439
8,2022-07-09,149,-48.0,197.0,-24.365482
9,2022-07-10,57,-92.0,149.0,-61.744966


In [42]:
df_ord_itm

,order_item_id,fk_product_id,fk_order_id,ordered_quantity,order_quantity_accepted,rate
0,62609,7818,119874,1,0,4993.00
1,62610,7607,119877,1,0,6203.40
2,62611,9916,119880,2,2,2904.41
3,62612,11118,119880,2,2,2205.00
4,62613,12702,119880,2,2,2200.85
...,...,...,...,...,...,...
20483,157584,10975,318966,1,1,12632.00
20484,157585,10975,318966,1,0,11814.00
20485,157586,10975,318966,1,1,10110.00
20486,157587,3284,318969,1,0,5271.00


In [43]:
orders = df_ord.merge(df_ord_itm, left_on = 'order_id', right_on = 'fk_order_id', how = 'left' )
orders

,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day,order_item_id,fk_product_id,fk_order_id,ordered_quantity,order_quantity_accepted,rate
0,119874,82228,5,Rejected,2021-07-01 00:35:28,2021,2021-07,2021-07-01,62609,7818,119874,1,0,4993.00
1,119877,40476,1,Rejected,2021-07-01 02:02:43,2021,2021-07,2021-07-01,62610,7607,119877,1,0,6203.40
2,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62611,9916,119880,2,2,2904.41
3,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62612,11118,119880,2,2,2205.00
4,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62613,12702,119880,2,2,2200.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20483,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157584,10975,318966,1,1,12632.00
20484,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157585,10975,318966,1,0,11814.00
20485,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157586,10975,318966,1,1,10110.00
20486,318969,122360,2,Rejected,2022-07-30 17:53:54,2022,2022-07,2022-07-30,157587,3284,318969,1,0,5271.00


In [44]:
orders = orders[orders['order_quantity_accepted'] > 0]
orders

,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day,order_item_id,fk_product_id,fk_order_id,ordered_quantity,order_quantity_accepted,rate
2,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62611,9916,119880,2,2,2904.41
3,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62612,11118,119880,2,2,2205.00
4,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62613,12702,119880,2,2,2200.85
8,119892,79316,4,Shipped,2021-07-01 03:19:14,2021,2021-07,2021-07-01,62617,8425,119892,2,2,3581.00
24,119907,45252,8,Shipped,2021-07-01 04:50:38,2021,2021-07,2021-07-01,62633,9924,119907,1,1,5021.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20467,318930,71772,5,Shipped,2022-07-30 13:25:25,2022,2022-07,2022-07-30,157568,12802,318930,10,10,2181.12
20482,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157583,10975,318966,1,1,17039.00
20483,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157584,10975,318966,1,1,12632.00
20485,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157586,10975,318966,1,1,10110.00


In [49]:
orders['Revenue'] = orders.order_quantity_accepted * orders.rate
orders

,order_id,fk_buyer_id,fk_depot_id,sales_order_status,creation_time,creation_year,creation_month,creation_day,order_item_id,fk_product_id,fk_order_id,ordered_quantity,order_quantity_accepted,rate,Revenue
2,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62611,9916,119880,2,2,2904.41,5808.82
3,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62612,11118,119880,2,2,2205.00,4410.00
4,119880,41728,8,Shipped,2021-07-01 02:42:12,2021,2021-07,2021-07-01,62613,12702,119880,2,2,2200.85,4401.70
8,119892,79316,4,Shipped,2021-07-01 03:19:14,2021,2021-07,2021-07-01,62617,8425,119892,2,2,3581.00,7162.00
24,119907,45252,8,Shipped,2021-07-01 04:50:38,2021,2021-07,2021-07-01,62633,9924,119907,1,1,5021.00,5021.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20467,318930,71772,5,Shipped,2022-07-30 13:25:25,2022,2022-07,2022-07-30,157568,12802,318930,10,10,2181.12,21811.20
20482,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157583,10975,318966,1,1,17039.00,17039.00
20483,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157584,10975,318966,1,1,12632.00,12632.00
20485,318966,167048,1,Shipped,2022-07-30 16:57:31,2022,2022-07,2022-07-30,157586,10975,318966,1,1,10110.00,10110.00


In [50]:
#Товары с самым высоким доходом
temp_table = orders[['fk_product_id', 'order_quantity_accepted', 'Revenue']]
temp_table = temp_table.groupby(by = 'fk_product_id').sum().reset_index()
temp_table.nlargest(487, 'Revenue')

,fk_product_id,order_quantity_accepted,Revenue
138,8219,1478,2.050146e+07
31,3610,936,1.124065e+07
359,12547,1572,9.716031e+06
77,7640,608,9.204148e+06
136,8210,652,8.645730e+06
...,...,...,...
453,14110,1,6.700000e+02
146,8316,1,5.890000e+02
86,7750,3,5.070000e+02
87,7756,14,3.597720e+02


In [51]:
#Топ-10 самых продаваемых товаров
temp_table = orders[['fk_product_id', 'order_quantity_accepted']]
temp_table = temp_table.groupby(by = 'fk_product_id').sum().reset_index()
temp_table.nlargest(10, 'order_quantity_accepted')

,fk_product_id,order_quantity_accepted
249,10235,1617
359,12547,1572
138,8219,1478
188,8444,1410
180,8428,1149
139,8221,1091
31,3610,936
360,12652,886
136,8210,652
77,7640,608


In [52]:
#Объем выручки по годам
total_sum = orders[['creation_year', 'Revenue']]
total_sum = total_sum.groupby(by = 'creation_year').sum().reset_index()
total_sum

,creation_year,Revenue
0,2021,6.031597e+07
1,2022,1.168582e+08
